In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

c:\Users\ahana\anaconda3\envs\mlflowenv\lib\site-packages\mlflow\utils\requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


In [9]:
import nltk

resources = [
    "stopwords",
    "punkt",
    "wordnet",
    "omw-1.4"
]

for r in resources:
    nltk.download(r)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ahana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ahana\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ahana\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ahana\AppData\Roaming\nltk_data...


In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
774,About the baby: Why wasn't big brother assumin...,negative
679,This show was so exhausting to watch and there...,positive
552,This movie was by far the worst movie I've eve...,negative
443,Watching QUINTET is not unlike watching a grou...,negative
915,"the film itself is absolutely brilliant, its t...",positive


In [10]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [11]:
df = normalize_text(df)
df.head()

,review,sentiment
774,baby big brother assuming hungry bottle nouris...,negative
679,show exhausting watch two number drowned world...,positive
552,movie far worst movie ever endure believe trie...,negative
443,watching quintet unlike watching group people ...,negative
915,film absolutely brilliant buzz rush make want ...,positive


In [12]:
df['sentiment'].value_counts()

sentiment
negative    255
positive    245
Name: count, dtype: int64

In [13]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [14]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
774,baby big brother assuming hungry bottle nouris...,0
679,show exhausting watch two number drowned world...,1
552,movie far worst movie ever endure believe trie...,0
443,watching quintet unlike watching group people ...,0
915,film absolutely brilliant buzz rush make want ...,1


In [15]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [16]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [19]:
print(X[0])

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 9 stored elements and shape (1, 100)>
  Coords	Values
  (0, 49)	1
  (0, 94)	1
  (0, 81)	1
  (0, 10)	6
  (0, 98)	1
  (0, 31)	1
  (0, 50)	1
  (0, 14)	1
  (0, 90)	1


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [3]:
import dagshub
import os
from dotenv import load_dotenv
import mlflow


# Load environment variables from .env file
load_dotenv()

mlflow_uri= os.getenv("mlflow_uri")
repo_owner= os.getenv("repo_owner")
repo_name= os.getenv("repo_name")

mlflow.set_tracking_uri(mlflow_uri)
dagshub.init(repo_owner=repo_owner, repo_name=repo_name, mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


c:\Users\ahana\anaconda3\envs\mlflowenv\lib\site-packages\mlflow\utils\requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


Accessing as AhanaKundu

Initialized MLflow to track repo "AhanaKundu/imdbproject_mlflow_endtoend"

Repository AhanaKundu/imdbproject_mlflow_endtoend initialized!

<Experiment: artifact_location='mlflow-artifacts:/61b0b6cab080490a8665011391aee477', creation_time=1755411198337, experiment_id='0', last_update_time=1755411198337, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [ ]:
#running experiment anf logging in mlflow required hyperparameters

In [22]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model1")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-08-17 11:52:08,839 - INFO - Starting MLflow run...
2025-08-17 11:52:09,063 - WARNING - Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /AhanaKundu/imdbproject_mlflow_endtoend.mlflow/api/2.0/mlflow/runs/create
2025-08-17 11:52:09,998 - INFO - Logging preprocessing parameters...
2025-08-17 11:52:11,730 - INFO - Initializing Logistic Regression model...
2025-08-17 11:52:11,730 - INFO - Fitting the model...
2025-08-17 11:52:11,746 - INFO - Model training complete.
2025-08-17 11:52:11,759 - INFO - Logging model parameters...
2025-08-17 11:52:12,549 - INFO - Making predictions...
2025-08-17 11:52:12,549 - INFO - Calculating evaluation metrics...
2025-08-17 11:52:12,563 - INFO - Logging evaluation metrics...
2025-08-17 11:52:14,594 - INFO - Saving and logging the model...
2025/08/17 11:52:23 WARNING mlflow.models.model: Input example should be provided to infer model signa

In [23]:
mlflow.end_run()